# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import json
import psycopg2
import pandas as pd
from sql_queries import *

In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()
# set the autocommit to true    
conn.set_session(autocommit=True)

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
song_files = get_files("data/song_data/")
print(f' Total number of song file : {len(song_files)}')

 Total number of song file : 71


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [10]:
result_songs = pd.read_json(song_files[7], lines=True)
result_songs= result_songs[['song_id','title','artist_id','year','duration']]
result_songs.tail()

,song_id,title,artist_id,year,duration
0,SOHKNRJ12A6701D1F8,Drop of Rain,AR10USD1187B99F3F1,0,189.57016


#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [12]:
for index,row in result_songs.iterrows():
    song_data=list(row.values)
    try:
        result=cur.execute(song_table_insert, song_data)
    except Exception as e:
        print(e)
    print(f'{len(song_data)} Records inserted successfully in songs table')

5 Records inserted successfully in songs table


Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [14]:
result_artists = pd.read_json(song_files[0], lines=True)
result_artists=result_artists[['artist_id','artist_latitude',\
                                             'artist_location','artist_longitude','artist_name']]
result_artists.tail()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name
0,ARGSJW91187B9B1D6B,35.21962,North Carolina,-80.01955,JennyAnyKind


#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [16]:
for index,row in result_artists.iterrows():
    artist_data=list(row.values)
    try:
        result=cur.execute(artist_table_insert, artist_data)
    except Exception as e:
        print(e)

cur.close()
conn.close()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [9]:
log_files = get_files("data/log_data/")
log_files[5:7]

['/home/saswata/personal/udacity nanodegree data engineer/project_postgresql/data/log_data/2018/11/2018-11-01-events.json',
 '/home/saswata/personal/udacity nanodegree data engineer/project_postgresql/data/log_data/2018/11/2018-11-09-events.json']

In [10]:
result_logs = pd.read_json(filepath, lines=True)
result_logs.dropna(subset=['userId', 'song','artist'],inplace=True)
print(f'Total number of records after removing null rows {result_logs.shape[0]}')                           
result_logs.head()

Total number of records after removing null rows 6820


,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
6,Facto Delafe y las flores azules,Logged In,Tegan,F,4,Levine,315.81995,paid,"Portland-South Portland, ME",PUT,NextSong,1.540794e+12,774,Enero en la playa,200,1542761878796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80
7,Kings Of Leon,Logged In,Kate,F,0,Harrell,204.27710,paid,"Lansing-East Lansing, MI",PUT,NextSong,1.540473e+12,671,Manhattan,200,1542761921796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",97
8,Franz Ferdinand,Logged In,Kate,F,1,Harrell,204.12036,paid,"Lansing-East Lansing, MI",PUT,NextSong,1.540473e+12,671,Michael,200,1542762125796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",97
9,Blue October,Logged In,Tegan,F,5,Levine,272.32608,paid,"Portland-South Portland, ME",PUT,NextSong,1.540794e+12,774,Drilled A Wire Through My Cheek,200,1542762193796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80
11,Elisa,Logged In,Kate,F,2,Harrell,248.97261,paid,"Lansing-East Lansing, MI",PUT,NextSong,1.540473e+12,671,Almeno Tu Nell'Universo,200,1542762329796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",97


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [11]:
df_time = result_logs[result_logs['page']=='NextSong']
df_time.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
6,Facto Delafe y las flores azules,Logged In,Tegan,F,4,Levine,315.81995,paid,"Portland-South Portland, ME",PUT,NextSong,1.540794e+12,774,Enero en la playa,200,1542761878796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80
7,Kings Of Leon,Logged In,Kate,F,0,Harrell,204.27710,paid,"Lansing-East Lansing, MI",PUT,NextSong,1.540473e+12,671,Manhattan,200,1542761921796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",97
8,Franz Ferdinand,Logged In,Kate,F,1,Harrell,204.12036,paid,"Lansing-East Lansing, MI",PUT,NextSong,1.540473e+12,671,Michael,200,1542762125796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",97
9,Blue October,Logged In,Tegan,F,5,Levine,272.32608,paid,"Portland-South Portland, ME",PUT,NextSong,1.540794e+12,774,Drilled A Wire Through My Cheek,200,1542762193796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80
11,Elisa,Logged In,Kate,F,2,Harrell,248.97261,paid,"Lansing-East Lansing, MI",PUT,NextSong,1.540473e+12,671,Almeno Tu Nell'Universo,200,1542762329796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",97


In [12]:
col_tmsp = pd.to_datetime(df_time['ts'], unit='ms')
col_start_time=col_tmsp.dt.strftime("%Y-%m-%d %H:%M:%S")
col_day=col_tmsp.dt.day
col_week=col_tmsp.dt.week
col_month=col_tmsp.dt.month
col_year=col_tmsp.dt.year
col_is_weekday=col_tmsp.dt.dayofweek

column_labels = ("start_time","day", "week", "month", "year", "weekday")
time_df = pd.concat([col_start_time,col_day,col_week,col_month,col_year, col_is_weekday],\
                    axis=1,ignore_index=True)
time_df.columns= column_labels
time_df.tail()

,start_time,day,week,month,year,weekday
8051,2018-11-14 23:18:31,14,46,11,2018,2
8052,2018-11-14 23:22:14,14,46,11,2018,2
8053,2018-11-14 23:26:53,14,46,11,2018,2
8054,2018-11-14 23:32:07,14,46,11,2018,2
8055,2018-11-14 23:38:06,14,46,11,2018,2


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [13]:
for i, row in time_df.iterrows():
    try:
        cur.execute(time_table_insert, list(row))
        # print('record inserted successfully')
    except Exception as e:
        print(e)

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [14]:
result_logs.tail()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
8051,Haddaway,Logged In,Chloe,F,11,Cuevas,223.97342,paid,"San Francisco-Oakland-Hayward, CA",PUT,NextSong,1.540941e+12,576,What Is Love,200,1542237511796,Mozilla/5.0 (Windows NT 5.1; rv:31.0) Gecko/20...,49
8052,The Black Crowes,Logged In,Chloe,F,12,Cuevas,279.79710,paid,"San Francisco-Oakland-Hayward, CA",PUT,NextSong,1.540941e+12,576,Sting Me,200,1542237734796,Mozilla/5.0 (Windows NT 5.1; rv:31.0) Gecko/20...,49
8053,Biz Markie,Logged In,Chloe,F,13,Cuevas,314.14812,paid,"San Francisco-Oakland-Hayward, CA",PUT,NextSong,1.540941e+12,576,This Is Something for the Radio,200,1542238013796,Mozilla/5.0 (Windows NT 5.1; rv:31.0) Gecko/20...,49
8054,Animal Collective,Logged In,Chloe,F,14,Cuevas,359.02649,paid,"San Francisco-Oakland-Hayward, CA",PUT,NextSong,1.540941e+12,576,Brother Sport,200,1542238327796,Mozilla/5.0 (Windows NT 5.1; rv:31.0) Gecko/20...,49
8055,Linkin Park,Logged In,Chloe,F,15,Cuevas,189.28281,paid,"San Francisco-Oakland-Hayward, CA",PUT,NextSong,1.540941e+12,576,Given Up (Album Version),200,1542238686796,Mozilla/5.0 (Windows NT 5.1; rv:31.0) Gecko/20...,49


In [15]:
user_df = result_logs[['userId','firstName','lastName','gender','level']]
# Drop any possible duplicates
user_df.drop_duplicates(['userId'],inplace=True)
# Remove records where first_name and last_name not present
user_df=user_df[(~user_df['userId'].isnull()) & (~user_df['firstName'].isnull()) & (~user_df['lastName'].isnull())]
user_df.reset_index(inplace=True,drop=True)
user_df.head()

/home/saswata/.virtualenvs/NewCastle/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,userId,firstName,lastName,gender,level
0,80,Tegan,Levine,F,paid
1,97,Kate,Harrell,F,paid
2,40,Tucker,Garrison,M,free
3,65,Amiya,Davidson,F,paid
4,26,Ryan,Smith,M,free


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [16]:
for i, row in user_df.iterrows():
    # There can be duplicate user ID as we're reading data from log files so handle exception
    try:
        cur.execute(user_table_insert, row)
    except psycopg2.IntegrityError as e:
        print('Duplicate user ID found, skipping insert')
    conn.commit()
print(f'{user_df.shape[0]} Records added successfully')

96 Records added successfully


Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [17]:
result_logs.columns

Index(['artist', 'auth', 'firstName', 'gender', 'itemInSession', 'lastName',
       'length', 'level', 'location', 'method', 'page', 'registration',
       'sessionId', 'song', 'status', 'ts', 'userAgent', 'userId'],
      dtype='object')

In [18]:
df_song_play = result_logs[['song','artist','length','ts','userId','level','location','userAgent','sessionId']]

df_song_play.drop_duplicates(subset=['ts','userId'],inplace=True,keep='first')

df_song_play.reset_index(drop=True,inplace=True)
print(f'Total num of records in song play dataframe : {df_song_play.shape[0]}')

df_song_play.head()

Total num of records in song play dataframe : 6820


/home/saswata/.virtualenvs/NewCastle/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,song,artist,length,ts,userId,level,location,userAgent,sessionId
0,Enero en la playa,Facto Delafe y las flores azules,315.81995,1542761878796,80,paid,"Portland-South Portland, ME","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",774
1,Manhattan,Kings Of Leon,204.27710,1542761921796,97,paid,"Lansing-East Lansing, MI","""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",671
2,Michael,Franz Ferdinand,204.12036,1542762125796,97,paid,"Lansing-East Lansing, MI","""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",671
3,Drilled A Wire Through My Cheek,Blue October,272.32608,1542762193796,80,paid,"Portland-South Portland, ME","""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",774
4,Almeno Tu Nell'Universo,Elisa,248.97261,1542762329796,97,paid,"Lansing-East Lansing, MI","""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",671


In [19]:
import traceback
count=0
for index, row in df_song_play.iterrows():
    # get songid and artistid from song and artist tables
    try:
        cur.execute(song_select, (row.song,row.artist, row.length))
        results = cur.fetchone()
        if results:
            song_id, artist_id = results
        else:
            song_id, artist_id = None, None
        row_tmsp = pd.to_datetime(row.ts, unit='ms').strftime("%Y-%m-%d %H:%M:%S")
        # insert songplay record
        songplay_data = (row_tmsp, row.userId, row.level, song_id, artist_id, row.sessionId, \
                        row.location, row.userAgent)
        #print(songplay_data)
        cur.execute(songplay_table_insert, songplay_data)
        count+=1
    except Exception as e:
        print(e)
        print(traceback.format_exc())
print(f'Total {count} records inserted successfully')

('2018-11-21 00:57:58', '80', 'paid', None, None, 774, 'Portland-South Portland, ME', '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"')
('2018-11-21 00:58:41', '97', 'paid', None, None, 671, 'Lansing-East Lansing, MI', '"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/37.0.2062.94 Safari/537.36"')
('2018-11-21 01:02:05', '97', 'paid', None, None, 671, 'Lansing-East Lansing, MI', '"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/37.0.2062.94 Safari/537.36"')
('2018-11-21 01:03:13', '80', 'paid', None, None, 774, 'Portland-South Portland, ME', '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"')
('2018-11-21 01:05:29', '97', 'paid', None, None, 671, 'Lansing-East Lansing, MI', '"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/37.0.2062.94 Safari/537.36"

('2018-11-21 03:34:27', '97', 'paid', None, None, 797, 'Lansing-East Lansing, MI', '"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/37.0.2062.94 Safari/537.36"')
('2018-11-21 03:35:17', '80', 'paid', None, None, 774, 'Portland-South Portland, ME', '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"')
('2018-11-21 03:39:23', '97', 'paid', None, None, 797, 'Lansing-East Lansing, MI', '"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/37.0.2062.94 Safari/537.36"')
('2018-11-21 03:39:46', '80', 'paid', None, None, 774, 'Portland-South Portland, ME', '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"')
('2018-11-21 03:42:23', '97', 'paid', None, None, 797, 'Lansing-East Lansing, MI', '"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/37.0.2062.94 Safari/537.36"

('2018-11-21 05:41:21', '65', 'paid', None, None, 64, 'Longview, TX', '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"')
('2018-11-21 05:41:30', '97', 'paid', None, None, 797, 'Lansing-East Lansing, MI', '"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/37.0.2062.94 Safari/537.36"')
('2018-11-21 05:45:50', '65', 'paid', None, None, 64, 'Longview, TX', '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"')
('2018-11-21 05:50:57', '12', 'free', None, None, 609, 'New York-Newark-Jersey City, NY-NJ-PA', 'Mozilla/5.0 (Windows NT 6.1; rv:31.0) Gecko/20100101 Firefox/31.0')
('2018-11-21 05:51:07', '97', 'paid', None, None, 797, 'Lansing-East Lansing, MI', '"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/37.0.2062.94 Safari/537.36"')
('2018-11-21 05:53:24', '65', 'paid', None, None, 64, 'Lo

('2018-11-21 08:52:18', '88', 'paid', None, None, 744, 'Sacramento--Roseville--Arden-Arcade, CA', '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"')
('2018-11-21 08:53:30', '36', 'paid', None, None, 728, 'Janesville-Beloit, WI', '"Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"')
('2018-11-21 08:57:22', '36', 'paid', None, None, 728, 'Janesville-Beloit, WI', '"Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"')
('2018-11-21 08:59:31', '88', 'paid', None, None, 744, 'Sacramento--Roseville--Arden-Arcade, CA', '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"')
('2018-11-21 09:01:02', '36', 'paid', None, None, 728, 'Janesville-Beloit, WI', '"Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safar

('2018-11-21 10:07:55', '15', 'paid', None, None, 764, 'Chicago-Naperville-Elgin, IL-IN-WI', '"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/36.0.1985.125 Chrome/36.0.1985.125 Safari/537.36"')
('2018-11-21 10:10:32', '88', 'paid', None, None, 744, 'Sacramento--Roseville--Arden-Arcade, CA', '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"')
('2018-11-21 10:11:48', '15', 'paid', None, None, 764, 'Chicago-Naperville-Elgin, IL-IN-WI', '"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/36.0.1985.125 Chrome/36.0.1985.125 Safari/537.36"')
('2018-11-21 10:12:32', '88', 'paid', None, None, 744, 'Sacramento--Roseville--Arden-Arcade, CA', '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"')
('2018-11-21 10:13:42', '26', 'free', None, None, 800, 'San Jose-Sunnyvale-Santa Cla

...

('2018-11-14 09:15:26', '49', 'paid', None, None, 568, 'San Francisco-Oakland-Hayward, CA', 'Mozilla/5.0 (Windows NT 5.1; rv:31.0) Gecko/20100101 Firefox/31.0')
('2018-11-14 09:17:19', '58', 'paid', None, None, 522, 'Augusta-Richmond County, GA-SC', '"Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"')
('2018-11-14 09:19:37', '58', 'paid', None, None, 522, 'Augusta-Richmond County, GA-SC', '"Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"')
('2018-11-14 09:22:56', '58', 'paid', None, None, 522, 'Augusta-Richmond County, GA-SC', '"Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"')
('2018-11-14 09:26:16', '58', 'paid', None, None, 522, 'Augusta-Richmond County, GA-SC', '"Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"')
('2018-11-14 09:26:21',

('2018-11-14 11:22:41', '29', 'paid', None, None, 559, 'Atlanta-Sandy Springs-Roswell, GA', '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.78.2 (KHTML, like Gecko) Version/7.0.6 Safari/537.78.2"')
('2018-11-14 11:23:20', '15', 'paid', None, None, 557, 'Chicago-Naperville-Elgin, IL-IN-WI', '"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/36.0.1985.125 Chrome/36.0.1985.125 Safari/537.36"')
('2018-11-14 11:26:55', '15', 'paid', None, None, 557, 'Chicago-Naperville-Elgin, IL-IN-WI', '"Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/36.0.1985.125 Chrome/36.0.1985.125 Safari/537.36"')
('2018-11-14 11:27:08', '29', 'paid', None, None, 559, 'Atlanta-Sandy Springs-Roswell, GA', '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.78.2 (KHTML, like Gecko) Version/7.0.6 Safari/537.78.2"')
('2018-11-14 11:30:54', '15', 'paid', None, None, 557, 'Chicago-Naperville-Elgin, IL-IN-WI', '"Mozilla

('2018-11-14 13:07:37', '4', 'free', None, None, 572, 'Parkersburg-Vienna, WV', '"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"')
('2018-11-14 13:08:45', '29', 'paid', None, None, 559, 'Atlanta-Sandy Springs-Roswell, GA', '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.78.2 (KHTML, like Gecko) Version/7.0.6 Safari/537.78.2"')
('2018-11-14 13:11:26', '34', 'free', None, None, 495, 'Milwaukee-Waukesha-West Allis, WI', 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; rv:31.0) Gecko/20100101 Firefox/31.0')
('2018-11-14 13:12:01', '29', 'paid', None, None, 559, 'Atlanta-Sandy Springs-Roswell, GA', '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.78.2 (KHTML, like Gecko) Version/7.0.6 Safari/537.78.2"')
('2018-11-14 13:12:28', '4', 'free', None, None, 572, 'Parkersburg-Vienna, WV', '"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"')
('2

('2018-11-14 16:39:20', '80', 'paid', None, None, 574, 'Portland-South Portland, ME', '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"')
('2018-11-14 16:42:10', '80', 'paid', None, None, 574, 'Portland-South Portland, ME', '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"')
('2018-11-14 16:42:22', '50', 'free', None, None, 578, 'New Haven-Milford, CT', '"Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"')
('2018-11-14 16:44:59', '80', 'paid', None, None, 574, 'Portland-South Portland, ME', '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"')
('2018-11-14 16:51:11', '80', 'paid', None, None, 574, 'Portland-South Portland, ME', '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Ge

('2018-11-14 19:02:02', '80', 'paid', None, None, 574, 'Portland-South Portland, ME', '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"')
('2018-11-14 19:05:18', '80', 'paid', None, None, 574, 'Portland-South Portland, ME', '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"')
('2018-11-14 19:09:00', '80', 'paid', None, None, 574, 'Portland-South Portland, ME', '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"')
('2018-11-14 19:12:39', '80', 'paid', None, None, 574, 'Portland-South Portland, ME', '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"')
('2018-11-14 19:18:13', '80', 'paid', None, None, 574, 'Portland-South Portland, ME', '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.3

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.